In [7]:
import json
import boto3
import random
import os
from concurrent.futures import ThreadPoolExecutor
import threading
RUNNING = True
threadLimiter = threading.BoundedSemaphore(6)
resultLock = threading.Lock()

s3 = boto3.resource('s3')

RAM_LIST = list(range(128, 3072 ,64))

In [ ]:
####################### TO BE CHANGED ###########################


source_path = 'dc-start-montage-0_25/'
folder_with_workflow_parts = 'montage-0_25/'
workflow_file = './workflow_montage_0.35.json'

###################### END TO BE CHANGED ####################### 

task_in_file_map = get_task_type_list(workflow_file)


for task_type in task_in_file_map:
    for in_file in task_in_file_map[task_type]:
        copy_source = {
            'Bucket': 'cegielskir',
            'Key': source_path + in_file
        }
        
        copy_file_from_to_bucket(copy_source, 'data-collection/' + folder_with_workflow_parts +task_type + '/' + in_file)

In [2]:
def get_task_type_list(workflow_file_path):
    with open(workflow_file_path) as result_json:
        data = json.load(result_json)
    task_type_input_files_map = {}
    for process in data['processes']:
        if process['name'] not in task_type_input_files_map:
            task_type_input_files_map[process['name']] = []
        for in_file in process['ins']:
            if data['signals'][in_file]['name'] not in task_type_input_files_map[process['name']]:
                task_type_input_files_map[process['name']].append(data['signals'][in_file]['name'])

    return task_type_input_files_map

In [1]:
def copy_file_from_to_bucket(copy_source, copy_destination):
    bucket = s3.Bucket('cegielskir')
    bucket.copy(copy_source, copy_destination)

In [2]:
def add_execution_result_to_json(task_id, result_dict, executionResultLock):
    executionResultLock.acquire()
    with open('./execution_data.json') as result_json:
        data = json.load(result_json)
    if 'montage_0.35' not in data:
        data['montage_0.35'] = []
    new_recort = {}
    
    new_recort['task_id'] = task_id
    for result_key in result_dict.keys():
        new_recort[result_key] = result_dict[result_key]
    
    data['montage_0.35'].append(new_recort)
    with open('./execution_data.json', 'w+') as result_json:
        json.dump(data, result_json, indent=4)
    executionResultLock.release()

In [3]:
def execute_workflow_task(config, ins, outs, ram, s3_path):
    options = {
        "bucket": "cegielskir",
        "prefix": s3_path
    }
    os.environ['FUNCTION_TYPE'] = str(ram)
    ins_arg = json.dumps(ins)
    outs_arg = json.dumps(outs)
    config_arg = json.dumps(config)
    options_arg = json.dumps(options)
    result = !node awsLambdaCommand.js {"'" + ins_arg + "'"} {"'" + outs_arg + "'"} {"'" + config_arg + "'"} {"'" + options_arg + "'"}
    return ' '.join(result)

In [ ]:
def run_single_task(filename, foldername)
    with open(foldername + filename) as single_workflow:
        data = json.load(single_workflow)
    task_name = data['processes'][0]['name']
    process = data['processes'][0]
    config = process['config']
    ins = [ data['signals'][i] for i in process['ins'] ]
    outs = [ data['signals'][i] for i in process['outs'] ]
    run_and_save_results(config, ins, outs, ram, filename.split('_')[0], filename)


In [ ]:
files = os.listdir('./montage-0_35/')
#files.reverse()
it = 1
while True:
    print("================================== NEW LOOP ====================================")
    if RUNNING:
        for file in files:
            for ram in RAM_LIST:
                task_core = file.split('_')[0]
                with open('./montage-0_35/' + file) as single_workflow:
                    data = json.load(single_workflow)
                task_name = data['processes'][0]['name']
                process = data['processes'][0]
                config = process['config']
                ins = [ data['signals'][i] for i in process['ins'] ]
                outs = [ data['signals'][i] for i in process['outs'] ]
                threadLimiter.acquire()
                print("Iteration: " + str(it))
                threading.Thread(target=run_and_save_results, args=(config, ins, outs, ram, task_core,file)).start()
                it += 1

================================== NEW LOOP ====================================
Iteration: 1
Starting mDiffFit_516.json with ram 128   core - mDiffFit
Iteration: 2
Starting mDiffFit_516.json with ram 192   core - mDiffFit
Iteration: 3
Starting mDiffFit_516.json with ram 256   core - mDiffFit
Iteration: 4
Starting mDiffFit_516.json with ram 320   core - mDiffFit
Iteration: 5
Starting mDiffFit_516.json with ram 384   core - mDiffFit
Iteration: 6
Starting mDiffFit_516.json with ram 448   core - mDiffFit
Iteration: 7
Starting mDiffFit_516.json with ram 512   core - mDiffFit
Iteration: 8
Starting mDiffFit_516.json with ram 576   core - mDiffFit
Iteration: 9
Starting mDiffFit_516.json with ram 640   core - mDiffFit
Iteration: 10
Starting mDiffFit_516.json with ram 704   core - mDiffFit
Iteration: 11
Starting mDiffFit_516.json with ram 768   core - mDiffFit
Iteration: 12
Starting mDiffFit_516.json with ram 832   core - mDiffFit
Iteration: 13
Starting mDiffFit_516.json with ram 896   core - m

Iteration: 110
Starting mDiffFit_8.json with ram 1216   core - mDiffFit
Iteration: 111
Starting mDiffFit_8.json with ram 1280   core - mDiffFit
Iteration: 112
Starting mDiffFit_8.json with ram 1344   core - mDiffFit
Iteration: 113
Starting mDiffFit_8.json with ram 1408   core - mDiffFit
Iteration: 114
Starting mDiffFit_8.json with ram 1472   core - mDiffFit
Iteration: 115
Starting mDiffFit_8.json with ram 1536   core - mDiffFit
Iteration: 116
Starting mDiffFit_8.json with ram 1600   core - mDiffFit
Iteration: 117
Starting mDiffFit_8.json with ram 1664   core - mDiffFit
Iteration: 118
Starting mDiffFit_8.json with ram 1728   core - mDiffFit
Iteration: 119
Starting mDiffFit_8.json with ram 1792   core - mDiffFit
Iteration: 120
Starting mDiffFit_8.json with ram 1856   core - mDiffFit
Iteration: 121
Starting mDiffFit_8.json with ram 1920   core - mDiffFit
Iteration: 122
Starting mDiffFit_8.json with ram 1984   core - mDiffFit
Iteration: 123
Starting mDiffFit_8.json with ram 2048   core - m

Iteration: 221
Starting mBackground_20.json with ram 2432   core - mBackground
Iteration: 222
Starting mBackground_20.json with ram 2496   core - mBackground
Iteration: 223
Starting mBackground_20.json with ram 2560   core - mBackground
Iteration: 224
Starting mBackground_20.json with ram 2624   core - mBackground
Iteration: 225
Starting mBackground_20.json with ram 2688   core - mBackground
Iteration: 226
Starting mBackground_20.json with ram 2752   core - mBackground
Iteration: 227
Starting mBackground_20.json with ram 2816   core - mBackground
Iteration: 228
Starting mBackground_20.json with ram 2880   core - mBackground
Iteration: 229
Starting mBackground_20.json with ram 2944   core - mBackground
Iteration: 230
Starting mBackground_20.json with ram 3008   core - mBackground
Iteration: 231
Starting mDiffFit_59.json with ram 128   core - mDiffFit
Iteration: 232
Starting mDiffFit_59.json with ram 192   core - mDiffFit
Iteration: 233
Starting mDiffFit_59.json with ram 256   core - mDi

Iteration: 330
Starting mProject_100.json with ram 576   core - mProject
Iteration: 331
Starting mProject_100.json with ram 640   core - mProject
Iteration: 332
Starting mProject_100.json with ram 704   core - mProject
Iteration: 333
Starting mProject_100.json with ram 768   core - mProject
Iteration: 334
Starting mProject_100.json with ram 832   core - mProject
Iteration: 335
Starting mProject_100.json with ram 896   core - mProject
Iteration: 336
Starting mProject_100.json with ram 960   core - mProject
Iteration: 337
Starting mProject_100.json with ram 1024   core - mProject
Iteration: 338
Starting mProject_100.json with ram 1088   core - mProject
Iteration: 339
Starting mProject_100.json with ram 1152   core - mProject
Iteration: 340
Starting mProject_100.json with ram 1216   core - mProject
Iteration: 341
Starting mProject_100.json with ram 1280   core - mProject
Iteration: 342
Starting mProject_100.json with ram 1344   core - mProject
Iteration: 343
Starting mProject_100.json wit

Iteration: 442
Starting mDiffFit_518.json with ram 1856   core - mDiffFit
Iteration: 443
Starting mDiffFit_518.json with ram 1920   core - mDiffFit
Iteration: 444
Starting mDiffFit_518.json with ram 1984   core - mDiffFit
Iteration: 445
Starting mDiffFit_518.json with ram 2048   core - mDiffFit
Iteration: 446
Starting mDiffFit_518.json with ram 2112   core - mDiffFit
Iteration: 447
Starting mDiffFit_518.json with ram 2176   core - mDiffFit
Iteration: 448
Starting mDiffFit_518.json with ram 2240   core - mDiffFit
Iteration: 449
Starting mDiffFit_518.json with ram 2304   core - mDiffFit
Iteration: 450
Starting mDiffFit_518.json with ram 2368   core - mDiffFit
Iteration: 451
Starting mDiffFit_518.json with ram 2432   core - mDiffFit
Iteration: 452
Starting mDiffFit_518.json with ram 2496   core - mDiffFit
Iteration: 453
Starting mDiffFit_518.json with ram 2560   core - mDiffFit
Iteration: 454
Starting mDiffFit_518.json with ram 2624   core - mDiffFit
Iteration: 455
Starting mDiffFit_518.j

Iteration: 555
Starting mBackground_106.json with ram 256   core - mBackground
Iteration: 556
Starting mBackground_106.json with ram 320   core - mBackground
Iteration: 557
Starting mBackground_106.json with ram 384   core - mBackground
Iteration: 558
Starting mBackground_106.json with ram 448   core - mBackground
Iteration: 559
Starting mBackground_106.json with ram 512   core - mBackground
Iteration: 560
Starting mBackground_106.json with ram 576   core - mBackground
Iteration: 561
Starting mBackground_106.json with ram 640   core - mBackground
Iteration: 562
Starting mBackground_106.json with ram 704   core - mBackground
Iteration: 563
Starting mBackground_106.json with ram 768   core - mBackground
Iteration: 564
Starting mBackground_106.json with ram 832   core - mBackground
Iteration: 565
Starting mBackground_106.json with ram 896   core - mBackground
Iteration: 566
Starting mBackground_106.json with ram 960   core - mBackground
Iteration: 567
Starting mBackground_106.json with ra

Iteration: 663
Starting mDiffFit_319.json with ram 1280   core - mDiffFit
Iteration: 664
Starting mDiffFit_319.json with ram 1344   core - mDiffFit
Iteration: 665
Starting mDiffFit_319.json with ram 1408   core - mDiffFit
Iteration: 666
Starting mDiffFit_319.json with ram 1472   core - mDiffFit
Iteration: 667
Starting mDiffFit_319.json with ram 1536   core - mDiffFit
Iteration: 668
Starting mDiffFit_319.json with ram 1600   core - mDiffFit
Iteration: 669
Starting mDiffFit_319.json with ram 1664   core - mDiffFit
Iteration: 670
Starting mDiffFit_319.json with ram 1728   core - mDiffFit
Iteration: 671
Starting mDiffFit_319.json with ram 1792   core - mDiffFit
Iteration: 672
Starting mDiffFit_319.json with ram 1856   core - mDiffFit
Iteration: 673
Starting mDiffFit_319.json with ram 1920   core - mDiffFit
Iteration: 674
Starting mDiffFit_319.json with ram 1984   core - mDiffFit
Iteration: 675
Starting mDiffFit_319.json with ram 2048   core - mDiffFit
Iteration: 676
Starting mDiffFit_319.j

Iteration: 771
Starting mDiffFit_578.json with ram 2304   core - mDiffFit
Iteration: 772
Starting mDiffFit_578.json with ram 2368   core - mDiffFit
Iteration: 773
Starting mDiffFit_578.json with ram 2432   core - mDiffFit
Iteration: 774
Starting mDiffFit_578.json with ram 2496   core - mDiffFit
Iteration: 775
Starting mDiffFit_578.json with ram 2560   core - mDiffFit
Iteration: 776
Starting mDiffFit_578.json with ram 2624   core - mDiffFit
Iteration: 777
Starting mDiffFit_578.json with ram 2688   core - mDiffFit
Iteration: 778
Starting mDiffFit_578.json with ram 2752   core - mDiffFit
Iteration: 779
Starting mDiffFit_578.json with ram 2816   core - mDiffFit
Iteration: 780
Starting mDiffFit_578.json with ram 2880   core - mDiffFit
Iteration: 781
Starting mDiffFit_578.json with ram 2944   core - mDiffFit
Iteration: 782
Starting mDiffFit_578.json with ram 3008   core - mDiffFit
Iteration: 783
Starting mDiffFit_87.json with ram 128   core - mDiffFit
Iteration: 784
Starting mDiffFit_87.json

Iteration: 883
Starting mDiffFit_183.json with ram 640   core - mDiffFit
Iteration: 884
Starting mDiffFit_183.json with ram 704   core - mDiffFit
Iteration: 885
Starting mDiffFit_183.json with ram 768   core - mDiffFit
Iteration: 886
Starting mDiffFit_183.json with ram 832   core - mDiffFit
Iteration: 887
Starting mDiffFit_183.json with ram 896   core - mDiffFit
Iteration: 888
Starting mDiffFit_183.json with ram 960   core - mDiffFit
Iteration: 889
Starting mDiffFit_183.json with ram 1024   core - mDiffFit
Iteration: 890
Starting mDiffFit_183.json with ram 1088   core - mDiffFit
Iteration: 891
Starting mDiffFit_183.json with ram 1152   core - mDiffFit
Iteration: 892
Starting mDiffFit_183.json with ram 1216   core - mDiffFit
Iteration: 893
Starting mDiffFit_183.json with ram 1280   core - mDiffFit
Iteration: 894
Starting mDiffFit_183.json with ram 1344   core - mDiffFit
Iteration: 895
Starting mDiffFit_183.json with ram 1408   core - mDiffFit
Iteration: 896
Starting mDiffFit_183.json wi

Iteration: 995
Starting mDiffFit_376.json with ram 1920   core - mDiffFit
Iteration: 996
Starting mDiffFit_376.json with ram 1984   core - mDiffFit
Iteration: 997
Starting mDiffFit_376.json with ram 2048   core - mDiffFit
Iteration: 998
Starting mDiffFit_376.json with ram 2112   core - mDiffFit
Iteration: 999
Starting mDiffFit_376.json with ram 2176   core - mDiffFit
Iteration: 1000
Starting mDiffFit_376.json with ram 2240   core - mDiffFit
Iteration: 1001
Starting mDiffFit_376.json with ram 2304   core - mDiffFit
Iteration: 1002
Starting mDiffFit_376.json with ram 2368   core - mDiffFit
Iteration: 1003
Starting mDiffFit_376.json with ram 2432   core - mDiffFit
Iteration: 1004
Starting mDiffFit_376.json with ram 2496   core - mDiffFit
Iteration: 1005
Starting mDiffFit_376.json with ram 2560   core - mDiffFit
Iteration: 1006
Starting mDiffFit_376.json with ram 2624   core - mDiffFit
Iteration: 1007
Starting mDiffFit_376.json with ram 2688   core - mDiffFit
Iteration: 1008
Starting mDiff

In [4]:
def run_and_save_results(config, ins, outs, ram, task_core,task_name):
    print("Starting " + str(task_name) + " with ram " + str(ram) + "   core - " + str(task_core))
    result = execute_workflow_task(config, ins, outs, ram, 'data-collection/montage-0_35/' + task_core)
    save_result(result, './montage-0.35-raw-results.json', str(ram), task_name)
    threadLimiter.release()


In [5]:
def save_result(result, filename,ram, task_id):
    resultLock.acquire()
    with open(filename) as result_json:
        data = json.load(result_json)
    if ram not in data:
        data[ram] = []
    result_dict = {}
    result_dict['task_id'] = task_id
    result_dict['result'] = str(result)
    data[ram].append(result_dict)
    
    with open(filename, 'w') as result_json:
        data = json.dump(data, result_json, indent=4)
    resultLock.release()